In [1]:
import numpy as np
import matplotlib.colors as colors
import matplotlib.pyplot as plt
# import seaborn as sns
import pandas as pd

import sklearn
import scipy

import sys
import os
import pathlib
import itertools
import glob
import re

from sklearn.preprocessing import QuantileTransformer, StandardScaler

package_path = os.path.join(pathlib.Path.home(), "mkvchain")
sys.path.append(package_path)
from model import FeatureDependentMarkovChain

In [2]:
root = pathlib.Path.home()
pt1 = os.path.join(root, 'research', 'GADS_Processed', "trajectories_with_features")

def glob_re(pattern, strings):
    return list(filter(re.compile(pattern).match, strings))

filenames = glob_re(r"gen_\d+_type_100_dtgrp_\d+_rating_\d+_state_Texas_raw.csv", os.listdir(pt1))

In [3]:
# len(filenames)

In [ ]:

mylist = []

for f in filenames[:20]:
    gen_id = int(re.findall(r"gen_(\d+)_.+", f)[0])

    data = pd.read_csv(os.path.join(pt1,f))
    data.set_index(pd.DatetimeIndex(data["x"]), inplace=True)
    data = data[~data.index.duplicated()]
    data = data[~data["y6"].isna()]
    data = data[~data["y7"].isna()]
    
    mylist.append(data)


In [19]:
data

,x,y1,y2,y3,ERCOT,y4,Pcp,Tmax,Tmin,y6,y7,y8
x,,,,,,,,,,,,
2013-05-08 12:00:00,2013-05-08T12:00:00.0,1.0,1.0,12182,36930.19272,128,2.42,85.2,59.8,0.0,8.0,5
2013-05-08 13:00:00,2013-05-08T13:00:00.0,1.0,1.0,12182,38239.14751,128,2.42,85.2,59.8,0.0,8.0,5
2013-05-08 14:00:00,2013-05-08T14:00:00.0,0.0,2.0,12182,39749.74598,128,2.42,85.2,59.8,0.0,9.0,5
2013-05-08 15:00:00,2013-05-08T15:00:00.0,0.0,2.0,12182,41124.88834,128,2.42,85.2,59.8,0.0,9.0,5
2013-05-08 16:00:00,2013-05-08T16:00:00.0,0.0,2.0,12182,42439.89312,128,2.42,85.2,59.8,0.0,9.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-31 20:00:00,2023-12-31T20:00:00.0,1.0,1.0,16071,44751.24000,365,1.75,64.3,39.4,579.0,30.0,12
2023-12-31 21:00:00,2023-12-31T21:00:00.0,1.0,1.0,16071,43892.82000,365,1.75,64.3,39.4,579.0,30.0,12
2023-12-31 22:00:00,2023-12-31T22:00:00.0,1.0,1.0,16071,43113.67000,365,1.75,64.3,39.4,579.0,30.0,12


In [ ]:

# features = d.loc[(~np.isnan(d["y6"])) & (~np.isnan(d["y5"])), ["y3", "y5", "y4", "y6"]]
# states = d.loc[features.index, "y2"].values

# print(sklearn.feature_selection.r_regression(features, states))
# print(sklearn.feature_selection.chi2(features, states))


In [20]:
states_new = []
features_new = []
lengths_new = []

for d in mylist:
    states = d["y2"].values.tolist()
    features = d[['y3', 'ERCOT', 'y4', 'Pcp', 'Tmax', 'Tmin', 'y6', 'y8', 'y7']].values
    l = len(d)

    states_new += [states]
    features_new += [features]
    lengths_new += [l]

states = np.concatenate(states_new).astype(int)
states -= 1
features = np.vstack(features_new)
lengths = np.array(lengths_new)

In [15]:
lengths.shape

(20,)

In [30]:
train_idx = 13 # int(lengths.size * .8)
val_idx = 17 # int(lengths.size * .9)

lengths_train = lengths[:train_idx]
lengths_val = lengths[train_idx:val_idx]
lengths_test = lengths[val_idx:]

states_train = states[:lengths_train.sum()]
states_val = states[lengths_train.sum():lengths_train.sum()+lengths_val.sum()]
states_test = states[lengths_train.sum()+lengths_val.sum():]

features_train = features[:lengths_train.sum()]
features_val = features[lengths_train.sum():lengths_train.sum()+lengths_val.sum()]
features_test = features[lengths_train.sum()+lengths_val.sum():]

In [29]:
features_train.shape, states_train.shape

((1077974, 9), (1077974,))

In [31]:

ss = StandardScaler()

features_train = ss.fit_transform(features_train)
features_val = ss.transform(features_val)
features_test = ss.transform(features_test)

In [32]:
n = 4
model1 = FeatureDependentMarkovChain(n, lam_frob=0, n_iter=1)
model1.fit(states_train, features_train*0, lengths_train, verbose=False)


IndexError: index 4 is out of bounds for axis 0 with size 4

In [ ]:

train1, val1, test1 = model1.score(states_train, features_train*0, lengths_train, average=False), \
    model1.score(states_val, features_val*0, lengths_val, average=False),
model1.score(states_test, features_test*0, lengths_test, average=False)
train1, val1, test1

In [ ]:
predictions = model1.predict(features_val)
for i, j in itertools.product(range(4), range(4)):
    y = predictions[:, i, j]
    val = len(np.unique(y))
    print(f"{i} -> {j} : {val}")

0 -> 0 : 2
0 -> 1 : 2
0 -> 2 : 2
0 -> 3 : 2
1 -> 0 : 2
1 -> 1 : 2
1 -> 2 : 2
1 -> 3 : 2
2 -> 0 : 2
2 -> 1 : 2
2 -> 2 : 2
2 -> 3 : 2
3 -> 0 : 2
3 -> 1 : 2
3 -> 2 : 2
3 -> 3 : 2


In [ ]:
from copy import deepcopy
train2, val2, test2 = -np.inf, -np.inf, -np.inf
best_lam = None
model2 = None
for lam in np.logspace(-3,-1,10):
    model = FeatureDependentMarkovChain(n, lam_frob=lam,  n_iter=1)
    model.As = deepcopy(model1.As)
    model.bs = deepcopy(model1.bs)
    model.fit(states_train, features_train, lengths_train, verbose=False, warm_start=True)
    traini, vali= model.score(states_train, features_train, lengths_train, average=False), \
          model.score(states_val, features_val, lengths_val, average=False)
        #   model.score(states_test, features_test, lengths_test, average=False)
    if vali > val2:
        train2 = traini
        val2 = vali
        # test2 = testi
        best_lam = lam
        model2 = model
train2, val2# test2

(np.float64(-762.7624227693096), np.float64(-584.0683045673397))

In [ ]:
model = FeatureDependentMarkovChain(n, lam_frob=best_lam, n_iter=1)
model.As = deepcopy(model1.As)
model.bs = deepcopy(model1.bs)
model.fit(states_train, features_train, lengths_train, verbose=False, warm_start=True)

In [ ]:
predictions2 = model.predict(features_val)
for i, j in itertools.product(range(4), range(4)):
    y = predictions2[:, i, j]
    val = len(np.unique(y.round(decimals=4)))
    print(f"{i} -> {j} : {val}")

0 -> 0 : 9
0 -> 1 : 4
0 -> 2 : 6
0 -> 3 : 2
1 -> 0 : 255
1 -> 1 : 263
1 -> 2 : 7
1 -> 3 : 2
2 -> 0 : 41
2 -> 1 : 2
2 -> 2 : 41
2 -> 3 : 2
3 -> 0 : 11
3 -> 1 : 2
3 -> 2 : 2
3 -> 3 : 11


In [ ]:
j = np.where(model.nonzero[0] == 0)[0][0]
A, b = model.As[0], model.bs[0]
feature_names = np.array(['y3', 'COAST', 'y4', 'PRCP', 'TMAX', 'TMIN', 'y6', 'y8'])
print(feature_names[np.argsort(A[:,j])[-5:]]) # month, TMAX', 'PRCP', age, load
print(feature_names[np.argsort(A[:,j])[:5]]) # time since last planned outage, doy, month

['y8' 'TMAX' 'PRCP' 'y3' 'COAST']
['y6' 'TMIN' 'y4' 'y8' 'TMAX']


In [ ]:
np.where(model.nonzero[0] == 0)[0][0]

np.int64(0)